# **Estadísticas básicas y correlación**

Se calcula la correlación estática

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Cargar el DataFrame
df = pd.read_csv('dataframe.csv', delimiter=",")
#df = df.drop(df.columns[[0, 15, 16, 17]], axis=1)

# Calcular las correlaciones
corr_pearson = df.corr(method='pearson')
corr_spearman = df.corr(method='spearman')
corr_kendall = df.corr(method='kendall')

# Seleccionar solo la correlación con la variable 'LowTemp' - es decir, la variable objetivo
corr_pearson_lowtemp = corr_pearson['LowTemp'].sort_values(ascending=False)
corr_spearman_lowtemp = corr_spearman['LowTemp'].sort_values(ascending=False)
corr_kendall_lowtemp = corr_kendall['LowTemp'].sort_values(ascending=False)

# Configurar el tamaño del gráfico
plt.figure(figsize=(15, 5))

# Crear mapas de calor para la correlación de Pearson, Spearman y Kendall con paleta de colores 'inferno'
plt.subplot(131)
sns.heatmap(corr_pearson_lowtemp.to_frame(), annot=True, cmap='inferno', cbar=False)
plt.title('Correlación de Pearson')

plt.subplot(132)
sns.heatmap(corr_spearman_lowtemp.to_frame(), annot=True, cmap='inferno', cbar=False)
plt.title('Correlación de Spearman')

plt.subplot(133)
sns.heatmap(corr_kendall_lowtemp.to_frame(), annot=True, cmap='inferno', cbar=False)
plt.title('Correlación de Kendall')

# Ajustar el diseño del gráfico
plt.tight_layout()

# Guardar la imagen en un archivo (puedes cambiar 'correlacion.png' al nombre que desees)
plt.savefig('correlacion.png')

# Mostrar el gráfico
plt.show()

Se calcula la correlación dinámica

In [ ]:
import pandas as pd

# Se ejecutan los análisis de correlación
df_results_p = pd.DataFrame()
df_results_s = pd.DataFrame()
df_results_k = pd.DataFrame()
text = ""

for i in range(24):
    text = str(i)
    print("Paso: " + text + " - Desplazamiento: " + str((-6 * i)))

    # Se crea la variable objetivo
    df[text] = df[df.columns[1]].shift(-6 * i)

    # Se ejecuta la correlacion de pearson
    corrP = df.corr('pearson')[[text]]  # Correlación
    corrP_rounded = corrP.round(6)  # Redondear a 6 cifras decimales
    df_results_p = pd.concat([df_results_p, corrP_rounded], axis=1)  # Se agregan los resultados al dataframe de resultados
    df_results_p = df_results_p.drop(df_results_p.index[-1])  # Se elimina el registro extra que se crea

    # Se ejecuta la correlacion de spearman
    corrS = df.corr('spearman')[[text]]  # Correlación
    corrS_rounded = corrS.round(6)  # Redondear a 6 cifras decimales
    df_results_s = pd.concat([df_results_s, corrS_rounded], axis=1)  # Se agregan los resultados al dataframe de resultados
    df_results_s = df_results_s.drop(df_results_s.index[-1])  # Se elimina el registro extra que se crea

    # Se ejecuta la correlacion de kendall
    corrK = df.corr('kendall')[[text]]  # Correlación
    corrK_rounded = corrK.round(6)  # Redondear a 6 cifras decimales
    df_results_k = pd.concat([df_results_k, corrK_rounded], axis=1)  # Se agregan los resultados al dataframe de resultados
    df_results_k = df_results_k.drop(df_results_k.index[-1])  # Se elimina el registro extra que se crea

    # Se elimina la variable objetiva creada en esta iteración
    # De esta manera se reduce el tiempo de ejecución de la correlación
    del df[text]

print("FIN")

Graficamos la correlación dinámica

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Supongamos que tus DataFrames se llaman df1_invertido, df2_invertido y df3_invertido
df1_invertido = pd.read_csv('resultados_pearson.csv', delimiter=",")
df2_invertido = pd.read_csv('resultados_spearman.csv', delimiter=",")
df3_invertido = pd.read_csv('resultados_kendall.csv', delimiter=",")

# Lista de variables que deseas comparar
variables_a_comparar = df1_invertido.columns[1:]

# Configuración del tamaño del gráfico y el diseño de subgráficos
fig, axs = plt.subplots(nrows=4, ncols=4, figsize=(20, 20))

# Iterar sobre las filas y columnas de la disposición
for i, row in enumerate(axs):
    for j, ax in enumerate(row):
        # Obtener el índice de la variable en la lista
        idx = i * 4 + j
        # Verificar si hemos excedido el número de variables a comparar
        if idx < len(variables_a_comparar):
            variable = variables_a_comparar[idx]
            ax.plot(df1_invertido['Hora'], df1_invertido[variable], label='Pearson')
            ax.plot(df2_invertido['Hora'], df2_invertido[variable], label='Spearman')
            ax.plot(df3_invertido['Hora'], df3_invertido[variable], label='Kendall')

            ax.set(xlabel='Hora')
            ax.set_title(variable, fontsize=16)  # Ajuste del tamaño del título

            # Mostrar los valores del eje x en vertical y con paso de 1 en 1
            ax.set_xticks(df1_invertido['Hora'][::1])
            ax.tick_params(axis='x', rotation=90)

            # Añadir cuadrícula para una mejor vista de los resultados
            ax.grid(True)

            # Añadir numeración en la esquina superior derecha o inferior derecha según el caso
            if idx + 1 in [9]:
                ax.text(0.95, 0.05, f'({idx+1})', transform=ax.transAxes, ha='right', va='bottom', fontsize=12, color='black', bbox=dict(facecolor='white', alpha=1))
            else:
                ax.text(0.95, 0.95, f'({idx+1})', transform=ax.transAxes, ha='right', va='top', fontsize=12, color='black', bbox=dict(facecolor='white', alpha=1))

        else:
            # Si excedemos el número de variables, ocultar el eje
            ax.axis('off')

# Ajustar el espaciado entre subgráficos
plt.tight_layout()

# Añadir la leyenda al gráfico principal
handles, labels = axs[0,0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 0), ncol=3, fontsize=14)

# Ajustar el layout para incluir la leyenda correctamente
plt.subplots_adjust(bottom=0.15)

#Se descarga el gráfico
plt.savefig('comparacion_correlacion.png')

# Mostrar el gráfico
plt.show()

Calculamos los promedios

In [ ]:
import pandas as pd

df1 = pd.read_csv('resultados_pearson.csv', delimiter=",")
df2 = pd.read_csv('resultados_spearman.csv', delimiter=",")
df3 = pd.read_csv('resultados_kendall.csv', delimiter=",")

# Lista de tus DataFrames
dataframes = [df1, df2, df3]

# Concatenar los DataFrames en uno solo
combined_df = pd.concat(dataframes, ignore_index=True)

# Calcular los promedios para cada columna
column_means = combined_df.mean()

# Crear un nuevo DataFrame con los resultados
result_df = pd.DataFrame(data=[column_means], columns=combined_df.columns)

# Redondear los valores a 6 cifras decimales
result_df_rounded = result_df.round(6)

# Mostrar el nuevo DataFrame con los promedios redondeados
print(result_df_rounded)

Integramos los dataframes

In [ ]:
import pandas as pd

df1 = pd.read_csv('df1.csv', delimiter=",")
df2 = pd.read_csv('df2.csv', delimiter=",")
df3 = pd.read_csv('df3.csv', delimiter=",")
df4 = pd.read_csv('dft.csv', delimiter=",")

# Lista de los DataFrames
dataframes = [df1, df2, df3, df4]

# Concatenar los DataFrames en uno solo
combined_df = pd.concat(dataframes, ignore_index=True)

# Mostrar el DataFrame combinado
print(combined_df)

Se grafican los promedios

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Configurar el tamaño del gráfico
plt.figure(figsize=(15, 20))

# Iterar sobre las columnas del DataFrame
for idx, (column_name, column_values) in enumerate(combined_df.iteritems(), 1):
    plt.subplot(5, 4, idx)

    # Calcular la mitad de los valores negativos y positivos
    neg_values = column_values[column_values < 0]
    pos_values = column_values[column_values >= 0]

    mid_neg = neg_values.max() / 2 if not neg_values.empty else 0
    mid_pos = pos_values.min() / 2 if not pos_values.empty else 0

    # Crear el gráfico de barras
    sns.barplot(x=column_values.index, y=column_values.values,
                hue=column_values.index, palette='viridis', legend=False, bottom=[mid_neg] * len(column_values))

    plt.title(f'{column_name}')
    plt.axhline(0, color='black', linewidth=0.8, linestyle='--')  # Línea horizontal en 0
    plt.xticks(rotation=45)  # Rotar las etiquetas del eje x para mayor legibilidad

    # Añadir cuadrícula para una mejor vista de los resultados
    plt.grid(True, axis='y')

    # Añadir numeración en la esquina superior derecha o inferior derecha según el caso
    plt.text(0.95, 0.05, f'({idx})', transform=plt.gca().transAxes, ha='right', va='bottom', fontsize=12, color='black', bbox=dict(facecolor='white', alpha=1))

    # Cambiar los índices en el eje x por etiquetas específicas
    plt.xticks(range(len(column_values)), ['Pearson', 'Spearman', 'Kendall', 'Totales'])

    # Establecer el nombre del eje x-y como una cadena vacía
    plt.ylabel('')
    plt.xlabel('')

# Ajustar el diseño del gráfico
plt.tight_layout()
plt.savefig('correlacion_var_guerrico.png')
plt.show()

Gráficos de dispersión y densidad

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Lee el archivo CSV en un DataFrame
file_path = "dataframe.csv"
df = pd.read_csv(file_path)

# Variables
variable_independiente = 'WindSpeed'
variable_objetivo = 'LowTemp'

# Crea el gráfico de dispersión
plt.figure(figsize=(10, 5))  # Se ajusta la proporción de aspecto

# Gráfico de dispersión
plt.subplot(1, 2, 1)
plt.scatter(df[variable_independiente], df[variable_objetivo], alpha=0.5)
plt.title(f'Diagrama de Dispersión: {variable_independiente} y {variable_objetivo}')
plt.xlabel(variable_independiente)
plt.ylabel(variable_objetivo)
plt.grid(True)
plt.xticks(rotation='vertical')
plt.xlim(df[variable_independiente].min() - 1, df[variable_independiente].max() + 2)
plt.ylim(df[variable_objetivo].min() - 1, df[variable_objetivo].max() + 2)
plt.xticks(np.arange(df[variable_independiente].min(), df[variable_independiente].max() + 1, 2))

# Ajusta los ticks en el eje Y
min_y = np.floor(df[variable_objetivo].min() / 5) * 5
max_y = np.ceil(df[variable_objetivo].max() / 5) * 5
plt.yticks(np.arange(min_y, max_y + 1, 5))

# Segundo gráfico: Gráfico de densidad
plt.subplot(1, 2, 2)
sns.kdeplot(data=df, x=variable_independiente, y=variable_objetivo, cmap="Blues", shade=True, bw_adjust=0.5)
plt.title(f'Diagrama de Densidad: {variable_objetivo} vs {variable_independiente}')
plt.xlabel(variable_independiente)
plt.ylabel(variable_objetivo)
plt.xticks(rotation='vertical')
plt.xticks(np.arange(df[variable_independiente].min(), df[variable_independiente].max() + 1, 2))

# Ajusta los ticks en el eje Y
plt.yticks(np.arange(min_y, max_y + 1, 5))

# Agregar líneas verticales para cada tick en el eje x
for tick in np.arange(df[variable_independiente].min(), df[variable_independiente].max() + 1, 2):
    plt.axvline(x=tick, color='gray', linestyle='--', linewidth=0.5)

# Agregar líneas horizontales para cada tick en el eje y
for tick in np.arange(min_y, max_y + 1, 5):
    plt.axhline(y=tick, color='gray', linestyle='--', linewidth=0.5)

plt.tight_layout()
plt.show()

Gráfico de lineas

In [ ]:
from matplotlib import pyplot as plt

# Dividir el eje x por 6 para mostrar el desplazamiento en horas
resultados['Desplazamiento_horas'] = resultados['Desplazamiento'] / 6

# Definir una lista de colores alternados
colors = ['blue', 'red']

# Graficar
plt.figure(figsize=(12, 4))
plt.plot(resultados['Desplazamiento_horas'], resultados[nombre], marker='o', linestyle='-')
plt.xlabel('Horas')
plt.ylabel('Velocidad del Viento (km/h)')

nombre = 'WindSpeed'

# Mostrar el valor de cada punto con colores alternados
for i, txt in enumerate(resultados[nombre]):
    x = resultados['Desplazamiento_horas'][i]
    y = resultados[nombre][i]
    color = colors[i % len(colors)]  # Alternar entre los colores de la lista

    if i == 13:
        plt.annotate(f'{txt:.2f}', (x, y), textcoords="offset points", xytext=(0,-20), ha='center', color=color)
    # Si el punto está muy cerca del borde superior, mueve el texto hacia abajo
    elif y > plt.ylim()[1] - 0.5:
        plt.annotate(f'{txt:.2f}', (x, y), textcoords="offset points", xytext=(0,-20), ha='center', color=color)
    else:
        plt.annotate(f'{txt:.2f}', (x, y), textcoords="offset points", xytext=(0,10), ha='center', color=color)

# Establecer ticks en el eje x para cada desplazamiento
plt.xticks(resultados['Desplazamiento_horas'])

# Configurar ticks en el eje y
#plt.yticks(range(int(min(resultados[nombre])) - 0, int(max(resultados[nombre])) + 4), step=2)

# Calcular los límites del eje y
y_min = int(min(resultados[nombre])) - 0
y_max = int(max(resultados[nombre])) + 10

# Crear una lista de valores de los ticks con la frecuencia deseada
y_ticks = list(range(y_min, y_max + 10, 5))

plt.grid(True)
plt.tight_layout()  # Ajustar automáticamente el diseño del gráfico para evitar superposiciones
plt.show()

Porcentaje de cambio para la correlación dinámica

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm

# Supongamos que estas son las rutas de tus archivos CSV
# Usamos los dataframes combinados
path_df1 = 'dataframe_1.csv'
path_df2 = 'dataframe_2.csv'

# Cargar DataFrames y descartar la cuarta fila
df1 = pd.read_csv(path_df1, delimiter=",")
df1 = df1.drop(df1.columns[[0,15,16]], axis=1)
df2 = pd.read_csv(path_df2, delimiter=",")
df2 = df2.drop(df2.columns[[0]], axis=1)

df1 = df1.drop(df1.index[-1])
df2 = df2.drop(df2.index[-1])

df2.columns = df1.columns

# Función para calcular el cambio porcentual
def calculate_percentage_change(corr1, corr2):
    return 100 * (corr2 - corr1) / np.abs(corr1)

# Calcular cambio porcentual para cada método
percentage_changes_pearson = calculate_percentage_change(df1.loc[0], df2.loc[0])
percentage_changes_spearman = calculate_percentage_change(df1.loc[1], df2.loc[1])
percentage_changes_kendall = calculate_percentage_change(df1.loc[2], df2.loc[2])

# Crear una lista de nombres de variables y ordenar alfabéticamente
variables = percentage_changes_pearson.index.tolist()
variables.sort()

# Crear una matriz para los valores de cada DataFrame
values_pearson = percentage_changes_pearson[variables].values
values_spearman = percentage_changes_spearman[variables].values
values_kendall = percentage_changes_kendall[variables].values

# Definir la cantidad de barras
num_bars = len(variables)

# Crear un rango de índices para las barras
bar_index = np.arange(num_bars)

# Ancho de cada barra
bar_width = 0.25

# Crear el gráfico de barras
plt.figure(figsize=(10, 6))

# Obtener la paleta de colores Pastel2
colors = cm.get_cmap('Pastel2')

# Barras para Pearson
bars1 = plt.bar(bar_index, values_pearson, width=bar_width, color=colors(0), label='Pearson')

# Barras para Spearman
bars2 = plt.bar(bar_index + bar_width, values_spearman, width=bar_width, color=colors(1), label='Spearman')

# Barras para Kendall
bars3 = plt.bar(bar_index + 2*bar_width, values_kendall, width=bar_width, color=colors(2), label='Kendall')

# Etiquetas en el eje x
plt.ylabel('Percent change')
plt.title('Percent change analysis between dataframe_1 and dataframe_2 (dynamic)')
plt.xticks(bar_index + bar_width, variables, rotation=90, ha='right')

# Agregar etiquetas a cada barra en la parte inferior
def add_labels(bars):
    for bar in bars:
        height = bar.get_height()
        plt.annotate('{}'.format(round(height, 2)),
                     xy=(bar.get_x() + bar.get_width() / 2, 0),  # coloca la etiqueta en la parte inferior
                     xytext=(0, 3),  # 3 puntos de desplazamiento vertical
                     textcoords="offset points",
                     ha='center', va='bottom', rotation=90)

add_labels(bars1)
add_labels(bars2)
#add_labels(bars3)

# Mostrar la leyenda en la parte inferior derecha
plt.legend(loc='lower right')

# Dibujar líneas punteadas entre las barras
for i in range(num_bars - 1):
    plt.axvline(x=bar_index[i] + bar_width * 3, linestyle='--', color='gray', linewidth=0.5)

# Reducir el espacio en blanco entre las barras y los límites del gráfico
plt.xlim(bar_index[0] - bar_width, bar_index[-1] + 0.75)  # Ajusta los límites del eje x

# Mostrar el gráfico
plt.tight_layout()
plt.show()

Calculamos el porcentaje de cambio de manera estática

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm

# Supongamos que estas son las rutas de tus archivos CSV
path_df1 = 'dataframe_1.csv'
path_df2 = 'dataframe_2.csv'

# Cargar DataFrames
df2.columns = df1.columns

# Función para calcular las correlaciones de un DataFrame
def calculate_correlations(df, target):
    corr_pearson = df.corr(method='pearson')[target].sort_values(ascending=False)
    corr_spearman = df.corr(method='spearman')[target].sort_values(ascending=False)
    corr_kendall = df.corr(method='kendall')[target].sort_values(ascending=False)
    return corr_pearson, corr_spearman, corr_kendall

# Calcular correlaciones para ambos DataFrames
corr_pearson1, corr_spearman1, corr_kendall1 = calculate_correlations(df1, 'LowTemp')
corr_pearson2, corr_spearman2, corr_kendall2 = calculate_correlations(df2, 'LowTemp')

# Función para calcular el cambio porcentual
def calculate_percentage_change(corr1, corr2):
    return 100 * (corr2 - corr1) / np.abs(corr1)

# Calcular cambio porcentual para cada método
percentage_changes_pearson = calculate_percentage_change(corr_pearson1, corr_pearson2)
percentage_changes_spearman = calculate_percentage_change(corr_spearman1, corr_spearman2)
percentage_changes_kendall = calculate_percentage_change(corr_kendall1, corr_kendall2)

Porcentaje de cambio para la correlación estática

In [ ]:
# Supongamos que estas son las rutas de tus archivos CSV
path_df1 = 'percentage_changes_pearson.csv'
path_df2 = 'percentage_changes_spearman.csv'
path_df3 = 'percentage_changes_spearman.csv'

# Cargar DataFrames
percentage_changes_pearson = pd.read_csv(path_df1, delimiter=",")
percentage_changes_spearman = pd.read_csv(path_df2, delimiter=",")
percentage_changes_kendall = pd.read_csv(path_df3, delimiter=",")

# Crear una lista de nombres de variables
variables = percentage_changes_pearson.index.tolist()

# Crear una matriz para los valores de cada DataFrame
values_pearson = percentage_changes_pearson.values
values_spearman = percentage_changes_spearman.values
values_kendall = percentage_changes_kendall.values

# Definir la cantidad de barras
num_bars = len(variables)

# Crear un rango de índices para las barras
bar_index = np.arange(num_bars)

# Ancho de cada barra
bar_width = 0.25

# Crear el gráfico de barras
plt.figure(figsize=(10, 6))

# Obtener la paleta de colores Pastel2
colors = cm.get_cmap('Pastel2')

# Barras para Pearson
bars1 = plt.bar(bar_index, values_pearson, width=bar_width, color=colors(0), label='Pearson')

# Barras para Spearman
bars2 = plt.bar(bar_index + bar_width, values_spearman, width=bar_width, color=colors(1), label='Spearman')

# Barras para Kendall
bars3 = plt.bar(bar_index + 2*bar_width, values_kendall, width=bar_width, color=colors(2), label='Kendall')

# Etiquetas en el eje x
plt.ylabel('Percent change')
plt.title('Percent change analysis between Villa Regina and Guerrico (original)')
plt.xticks(bar_index + bar_width, variables, rotation=90, ha='right')

# Agregar etiquetas a cada barra en la parte inferior
def add_labels(bars):
    for bar in bars:
        height = bar.get_height()
        plt.annotate('{}'.format(round(height, 2)),
                     xy=(bar.get_x() + bar.get_width() / 2, 0),  # coloca la etiqueta en la parte inferior
                     xytext=(0, 3),  # 3 puntos de desplazamiento vertical
                     textcoords="offset points",
                     ha='center', va='bottom', rotation=90)

add_labels(bars1)
add_labels(bars2)
add_labels(bars3)

# Mostrar la leyenda en la parte inferior derecha
plt.legend(loc='lower right')

# Dibujar líneas punteadas entre las barras
for i in range(num_bars - 1):
    plt.axvline(x=bar_index[i] + bar_width * 3.15, linestyle='--', color='gray', linewidth=0.5)

# Reducir el espacio en blanco entre las barras y los límites del gráfico
plt.xlim(bar_index[0] - bar_width, bar_index[-1] + 0.75)  # Ajusta los límites del eje x

# Mostrar el gráfico
plt.tight_layout()
plt.show()

Porcentaje de cambio (velocidad del viento, aire desplazado y promedio de lluvia)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# Leer el archivo CSV
df1 = pd.read_csv('dataframe.csv', delimiter=';') #usamos el dataframe combinado

# Las primeras tres columnas son: Columna de referencia (eje x) y tres columnas con porcentajes de cambio
x_values = df1.iloc[:, 0]  # Columna de referencia para el eje x
y_values = df1.iloc[:, 1:4]  # Columnas con porcentajes de cambio

# Calcular el promedio de las columnas de porcentaje de cambio
y_values['Promedio'] = y_values.mean(axis=1)

# Obtener el mapa de colores
color_map = cm.get_cmap('Pastel2')

# Graficar un gráfico de barras para cada columna de porcentaje de cambio
fig, axes = plt.subplots(2, 2, figsize=(8, 10))
axes = axes.flatten()  # Convertir la matriz de ejes en un arreglo unidimensional

for i, column in enumerate(y_values.columns):
    bars = axes[i].bar(x_values, y_values[column], color=color_map(i))
    axes[i].set_xlabel('Hours')  # Reemplazar 'Eje X' con el nombre de tu primera columna si es necesario
    axes[i].set_ylabel('Percent change (%)')
    axes[i].set_title(f'Percent change - {column}')
    axes[i].set_xticks(x_values)
    axes[i].tick_params(axis='x', rotation=90)  # Rotar etiquetas del eje x para mejorar la legibilidad
    axes[i].grid(True)

    # Agregar los valores de las barras
    for bar, height in zip(bars, y_values[column]):
        va = 'bottom' if 0 <= height < 62 else 'top'
        axes[i].text(bar.get_x() + bar.get_width() / 2, height, f'{height:.2f}', ha='center', va=va, color='black', rotation=90)

# Graficar el promedio de los porcentajes de cambio
bars = axes[3].bar(x_values, y_values['Promedio'], color=color_map(3))
axes[3].set_xlabel('Hours')
axes[3].set_ylabel('Percent change (%)')
axes[3].set_title('Percent change')
axes[3].set_xticks(x_values)
axes[3].tick_params(axis='x', rotation=90)
axes[3].grid(True)

# Agregar los valores de las barras para el promedio
for bar, height in zip(bars, y_values['Promedio']):
    va = 'bottom' if 0 <= height < 62 else 'top'
    axes[3].text(bar.get_x() + bar.get_width() / 2, height, f'{height:.2f}', ha='center', va=va, color='black', rotation=90)

plt.tight_layout()
plt.show()